In [1]:
import pandas as pd
import numpy as np
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import time
import threading
from threading import Timer
import argparse
from datetime import datetime, timedelta, date, time

In [ ]:
from data_preparation import *
from price_prediction import *
from trading_strategy import *

In [4]:
###########################################Set hyperparameters###################################################

# Create the parser
parser = argparse.ArgumentParser(description="argument parser for preprocessing and downloading data")

#timing intervals
parser.add_argument("--start_date", type=str, default = None, help="start date of all data")
parser.add_argument("--days", type=int, default = 500, help="number of days needed")
parser.add_argument("--room_na", type=int, default = 200, help="room of na days")
parser.add_argument("--interval", type=str, default = "1d", choices=["1d","1wk","1mo"], help="choose an interval")
parser.add_argument("--end_date", type=str, default = str(date.today()) , help="end date of all data")

#stock needed
parser.add_argument("--stock", action="store_true", default = 'AAPL' , help = "stock to consider download")

#log indicaters
parser.add_argument("--need_log", action="store_false", help="check if log transformation needed")

#finta indicaters without params
parser.add_argument("--need_ta", action="store_false", help="check if ta indicaters needed")
parser.add_argument("--choices_ta", default = "['RSI','STOCH']", help="choices of ta indicaters")

#trend indicaters
parser.add_argument("--need_macd", action="store_false", help="check if macd indicaters needed")
parser.add_argument("--columns_macd", default = "['Close']" , help="target column of macd")
parser.add_argument("--short_span", type = int, default = 12 , help="short span of macd")
parser.add_argument("--long_span", type = int, default = 26 , help="long span of macd")
parser.add_argument("--signal_span", type = int, default = 9 , help="signal span of macd")

#ema indicaters
parser.add_argument("--need_ema", action="store_false", help="check if ema indicaters needed")
parser.add_argument("--days_ema", default = "[5,15,25,50]" , help="days of ema installed")
parser.add_argument("--columns_ema", default = "['Close']" , help="target column of ema")

#prev day
parser.add_argument("--need_prev", action="store_false", help="check if previous day indicaters needed")
parser.add_argument("--columns_prev", default = "['Close']" , help="target column of previous day")
parser.add_argument("--days_prev", type = int, default = "20" , help= "number of previous days needed")

#shift target
parser.add_argument("--shift_target", action="store_false", help="check if shifting target for backtesting is needed")
#task definition
parser.add_argument("--regression", action = 'store_false', help="to do regression or classification task")
parser.add_argument("--training_size", type = int, default = 300, help = "number of days to used for traning the model" )
parser.add_argument("--training_interval", type = int,  default = 1, help = 'interval to train a new model for each testing day')

#data preprocessing
parser.add_argument("--return_normalization", action = 'store_true', help="to do return rate normalization or not")
parser.add_argument("--min_max_normalization", action = 'store_true', help="to do min max normalization or not" )
parser.add_argument("--standard_normalization", action = 'store_false', help = "to do standard normalization or not"  )

#sequential
parser.add_argument("--sequential", action = 'store_true', help="using sequential model or  not")
parser.add_argument("--window_size", type = int, default = 20, help="using sequential model or  not")
parser.add_argument("--model", default = 'Conv', help="model to use for training")
parser.add_argument("--model_param", default = {}, help = 'model hyperparamters')



In [2]:
def GetPosition():
    class ib_class(EWrapper, EClient):

        def __init__(self):
            EClient.__init__(self, self)

            self.all_positions = pd.DataFrame([], columns = ['Account','Symbol', 'Quantity', 'Average Cost', 'Sec Type'])
            self.reqPositions()

        def error(self, reqId, errorCode:int, errorString:str):
            if reqId > -1:
                print("Error. Id: " , reqId, " Code: " , errorCode , " Msg: " , errorString)

        def position(self, account, contract, pos, avgCost):
                index = str(account)+str(contract.symbol)
                self.all_positions.loc[index]= account, contract.symbol, pos, avgCost, contract.secType

    def run_loop():
        app.run()

    app = ib_class()
    app.connect('127.0.0.1', 7497, 1010)
    #Start the socket in a thread
    api_thread = threading.Thread(target=run_loop, daemon=True)
    api_thread.start()
    time.sleep(1) #Sleep interval to allow time for connection to server

    app.reqPositions() # associated callback: position
    print("Waiting for IB's API response for accounts positions requests...\n")
    time.sleep(3)
    current_positions = app.all_positions
    current_positions.set_index('Account',inplace=True,drop=True) #set all_positions DataFrame index to "Account"
    
    return current_positions

In [4]:
pos = GetPosition()
pos

Waiting for IB's API response for accounts positions requests...



,Symbol,Quantity,Average Cost,Sec Type
Account,,,,
DU7662291,AAPL,4.0,179.5325,STK


In [2]:
def PlaceOrder(ticker = "AAPL", direction = "BUY", quantity = 1):
    class PlaceOrderClass(EClient, EWrapper):
        def __init__(self):
            EClient.__init__(self, self)

        def nextValidId(self, orderId):

            mycontract = Contract()
            mycontract.symbol = ticker
            mycontract.secType = "STK"   
            mycontract.exchange = "SMART"
            mycontract.currency = "USD"

            self.reqContractDetails(orderId, mycontract)

        def contractDetails(self, reqId: int, contractDetails):
            print(contractDetails.contract)

            myorder = Order()
            myorder.orderId = reqId
            myorder.action = direction
            myorder.tif = "GTC"
            myorder.eTradeOnly = ""
            myorder.firmQuoteOnly = ""
            myorder.orderType = "MKT"
            #myorder.lmtPrice = 45.72
            myorder.totalQuantity = quantity

            self.placeOrder(myorder.orderId, contractDetails.contract, myorder)


        def openOrder(self, orderId, contract: Contract, order: Order, orderState):
            print(f"openOrder. orderId: {orderId}, contract: {contract}, order: {order}")

        def orderStatus(self, orderId, status: str, filled: int, remaining: int, avgFillPrice: float, permId: int, parentId: int, lastFillPrice: float, clientId: int, whyHeld: str, mktCapPrice: float):
            print(f"orderId: {orderId}, status: {status}, filled: {filled}, remaining: {remaining}, avgFillPrice: {avgFillPrice}, permId: {permId}, parentId: {parentId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice: {mktCapPrice}")

        def execDetails(self, reqId: int, contract: Contract, execution):
            print(f"reqId: {reqId}, contract: {contract}, execution: {execution}")

 
    app = PlaceOrderClass()
    app.connect("127.0.0.1", 7497, 9)
    app.run()
    

In [ ]:
PlaceOrder(ticker = "AAPL", direction = "BUY", quantity = 1)

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


265598,AAPL,STK,,0.0,,,SMART,NASDAQ,USD,AAPL,NMS,False,,combo:
openOrder. orderId: 13, contract: 265598,AAPL,STK,,0.0,?,,SMART,,USD,AAPL,NMS,False,,combo:, order: 13,9,2081959988: MKT BUY 1@176.070000 GTC
orderId: 13, status: Submitted, filled: 0.0, remaining: 1.0, avgFillPrice: 0.0, permId: 2081959988, parentId: 0, lastFillPrice: 0.0, clientId: 9, whyHeld: , mktCapPrice: 0.0
reqId: -1, contract: 265598,AAPL,STK,,0.0,,,ISLAND,,USD,AAPL,NMS,False,,combo:, execution: ExecId: 00012ec5.64e75c82.01.01, Time: 20230824  17:50:29, Account: DU7662291, Exchange: ISLAND, Side: BOT, Shares: 1.000000, Price: 176.040000, PermId: 2081959988, ClientId: 9, OrderId: 13, Liquidation: 0, CumQty: 1.000000, AvgPrice: 176.040000, OrderRef: , EvRule: , EvMultiplier: 0.000000, ModelCode: , LastLiquidity: 1
openOrder. orderId: 13, contract: 265598,AAPL,STK,,0.0,?,,SMART,,USD,AAPL,NMS,False,,combo:, order: 13,9,2081959988: MKT BUY 1@176.070000 GTC
orderId: 13, status: Filled, filled: 1.0, remaining: 0.0, avgFill

ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 2103 Market data farm connection is broken:cashfarm
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2103 Market data farm connection is broken:usfarm.nj
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2157 Sec-def data farm connection is broken:secdefil
ERROR -1 2105 HMDS data farm connection is broken:ushmds
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 1102 Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; usfuture; cashfarm; usfarm; ushmds; secdefil.
ERROR -1 2103 Market data farm conn

ERROR -1 2103 Market data farm connection is broken:cashfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2103 Market data farm connection is broken:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2105 HMDS data farm connection is broken:ushmds
ERROR -1 2157 Sec-def data farm connection is broken:secdefil
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 2103 Market data farm connection is broken:usfarm.nj
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2103 Market data farm connection is broken:cashfarm
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2157 Sec-def data farm connection is broken:secdefil
ERROR -1 2105 HMDS data farm connection is broken:ushmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data far

In [ ]:
def main(args):
    #data preparation, traning, strategy
    data = data_downloader(args)
    prediction = predict(args,data)
    direction = trading_decision(prediction,test_day) #-1 for short, 1 for long, 0 for doing nothing
    
    pos = GetPosition()
    current_position = pos["Position"]
    
    if direction == 1 and current_position > 0: #stay long position
        print('stay long position')
        
    elif direction == -1 and current_position < 0: #stay short position
        print('stay short position')
        
    elif direction == 1 and current_position < 0: #close short position, open long position
        print('close short position, open long position')
        PlaceOrder(ticker = args.stock, direction = "BUY", quantity = abs(current_position)*2)
        
    elif direction == -1 and current_position > 0: #close long position, open short position
        print('close long position, open short position')
        PlaceOrder(ticker = args.stock, direction = "SELL", quantity = abs(current_position)*2)
    
    else: #no trading
        print('error: except condition of no trading')
        continue

In [ ]:
if __name__ == '__main__':
    args = parser.parse_args([])
    main(args)